In [11]:
video_path = "data/TestVideo/UAV and hands.mp4"

In [13]:
!pip install keras_cv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/650.7 kB ? eta -:--:--
   --------------------------------------- 650.7/650.7 kB 20.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/950.8 kB ? eta -:--:--
   --------------------------------------- 950.8/950.8 kB 35.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   ---------------------------------------- 5.3/5.3 MB 55.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21545 sha256=40e962f15914c7a38970a950a01351f7102fa1ce7eb4655f9cf78f63c6747882
  Stored in directory: C:\Users\Lenovo\AppData\Local\Temp\pip-ephem-wheel-cache-3x_1rvm1\wheels\8f\46\1c\1f4e5d73a20eb816ead5014e97cdeb3928cf314fc46c7bab61
Successfully built promise

   ----- ----------------------------------

  DEPRECATION: Building 'promise' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'promise'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [10]:
!pip install opencv-python tensorflow 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   --------- ------------------------------ 9.4/39.0 MB 54.8 MB/s eta 0:00:01
   ----------------------- ---------------- 22.5/39.0 MB 60.8 MB/s eta 0:00:01
   ------------------------------------ --- 35.7/39.0 MB 62.2 MB/s eta 0:00:01
   ---------------------------------------- 39.0/39.0 MB 59.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   - -------------------------------------- 14.7/332.0 MB 70.0 MB/s eta 0:00:05
   --- ------------------------------------ 30.4/332.0 MB 73.1 MB/s eta 0:00:05
   ----- ---------------------------------- 46.4/332.0 MB 73.4 MB/s eta 0:00:04
   ------- -------------------------------- 60.3/332.0 MB 72.0 MB/s eta 0:00:04
   -------- ------------------------------- 73.7/332.0 MB 70.7 MB/s eta 0:00:04
   ---------- ----------------------------- 84.9/332.0 MB 68.7 MB/s eta

In [11]:
!pip install -U ultralytics

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 29.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/805.6 kB ? eta -:--:--
   --------------------------------------- 805.6/805.6 kB 93.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.0 MB ? eta -:--:--
   ---------- ----------------------------- 12.3/45.0 MB 58.8 MB/s eta 0:00:01
   ---------------------- ----------------- 25.7/45.0 MB 62.5 MB/s eta 0:00:01
   ---------------------------------- ----- 39.3/45.0 MB 64.9 MB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 MB 64.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---- ----------------------------------- 13.6/110.9 MB 67.3 MB/s eta 0:00:02
   --------- ------------------------------ 27.0/110.9 MB 67.2 MB/s eta 0:00:02
   ----------

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import keras
import keras_cv
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cosine
import os
from collections import deque, defaultdict
import warnings
from ultralytics import YOLO
import math

In [ ]:


# Игнорируем предупреждения
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
os.environ["OPENCV_VIDEOIO_PRIORITY_MSMF"] = "0"
cv2.setLogLevel(0)

# Загрузка обученного детектора
model = YOLO('C:/ИТМО/НИР 3/yolo simlpe 8n.pt')

# Загружаем EfficientNet для извлечения эмбеддингов
efficientnet = keras_cv.models.EfficientNetV2Backbone.from_preset("efficientnetv2_s")
print("EfficientNetV2Backbone загружена")

# Получаем имена классов из модели YOLO
class_names = model.names if hasattr(model, 'names') else {0: 'object'}

In [ ]:
#Предобрабодка видео для дальнейшей 
def preprocess_roi(roi):
    """Preprocess ROI for EfficientNet input"""
    if roi is None or roi.size == 0:
        return None
    
    # Проверка минимального размера перед обработкой
    if roi.shape[0] <= 10 or roi.shape[1] <= 10:
        return None
        
    # Увеличение маленьких ROI
    if roi.shape[0] < 30 or roi.shape[1] < 30:
        pad = 15
        h, w = roi.shape[:2]
        if h > 0 and w > 0:
            padded = np.zeros((h + 2*pad, w + 2*pad, 3), dtype=roi.dtype)
            padded[pad:pad+h, pad:pad+w] = roi
            roi = padded
    
    roi = tf.image.resize(roi, (224, 224))
    roi = tf.keras.applications.efficientnet.preprocess_input(roi)
    return roi


def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors with normalization"""
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return np.dot(a, b)


def calculate_iou(box1, box2):
    """Calculate IoU between two bounding boxes [x1, y1, x2, y2]"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0


In [ ]:
def calculate_velocity_similarity(prev_box, curr_box, prev_velocity, frame_rate=30):
    """
    Calculate similarity based on velocity consistency
    Returns a score between 0 and 1 where 1 is perfect consistency
    """
    # Calculate current velocity
    prev_center = ((prev_box[0] + prev_box[2]) // 2, (prev_box[1] + prev_box[3]) // 2)
    curr_center = ((curr_box[0] + curr_box[2]) // 2, (curr_box[1] + curr_box[3]) // 2)
    
    curr_velocity = (
        (curr_center[0] - prev_center[0]) * frame_rate,
        (curr_center[1] - prev_center[1]) * frame_rate
    )
    
    if prev_velocity is None:
        return 1.0  # No previous velocity to compare with
    
    # Calculate angle difference between velocities
    dot_product = prev_velocity[0] * curr_velocity[0] + prev_velocity[1] * curr_velocity[1]
    norm_prev = math.sqrt(prev_velocity[0]**2 + prev_velocity[1]**2)
    norm_curr = math.sqrt(curr_velocity[0]**2 + curr_velocity[1]**2)
    
    if norm_prev == 0 or norm_curr == 0:
        return 0.5  # Neutral score when one velocity is zero
    
    cos_theta = dot_product / (norm_prev * norm_curr + 1e-8)
    angle_similarity = (cos_theta + 1) / 2  # Normalize to [0, 1]
    
    # Calculate magnitude similarity
    magnitude_ratio = min(norm_prev, norm_curr) / max(norm_prev, norm_curr) if max(norm_prev, norm_curr) > 0 else 1.0
    
    # Combined velocity similarity
    return 0.7 * angle_similarity + 0.3 * magnitude_ratio

In [ ]:
class EmbOCTracker:
    
    def __init__(self, max_age=30, min_hits=3, iou_threshold=0.3, 
                 embedding_threshold=0.7, recovery_window=15, 
                 trajectory_length=50, frame_rate=30, 
                 velocity_weight=0.3, observation_confidence_alpha=0.7):
        self.tracks = []  # Список активных треков
        self.next_id = 1  # Следующий ID для новых треков
        self.max_age = max_age  # Макс. кадров без обнаружения перед удалением
        self.min_hits = min_hits  # Мин. подтверждений для активного трека
        self.iou_threshold = iou_threshold
        self.embedding_threshold = embedding_threshold
        self.recovery_window = recovery_window  # Окно для восстановления ID
        self.trajectory_length = trajectory_length
        self.lost_tracks = []  # Временно потерянные треки
        self.frame_count = 0
        self.frame_rate = frame_rate
        self.trajectories = defaultdict(lambda: deque(maxlen=trajectory_length))
        
        # Observation-Centric параметры
        self.velocity_weight = velocity_weight  # Вес скорости в комбинированной метрике
        self.observation_confidence_alpha = observation_confidence_alpha  # Параметр для адаптивного доверия наблюдениям
        self.history_length = 10  # Длина истории для оценки нелинейности движения

    def estimate_observation_confidence(self, track, detection_box):
        """
        Observation-Centric: Оценка надежности наблюдения для конкретного трека
        Возвращает значение от 0 (низкая надежность) до 1 (высокая надежность)
        """
        # 1. Анализ согласованности с предсказанным положением
        predicted_box = track.get('predicted_bbox', track['bbox'])
        iou_consistency = calculate_iou(predicted_box, detection_box)
        
        # 2. Анализ истории движения для оценки линейности
        trajectory = self.trajectories[track['track_id']]
        if len(trajectory) > self.history_length:
            # Оцениваем линейность движения по истории
            points = list(trajectory)[-self.history_length:]
            linear_confidence = self._evaluate_motion_linearity(points)
        else:
            linear_confidence = 0.8  # По умолчанию предполагаем линейное движение
            
        # 3. Анализ скорости и ускорения
        velocity_confidence = 1.0
        if 'velocity_history' in track and len(track['velocity_history']) > 2:
            velocity_confidence = self._evaluate_velocity_stability(track['velocity_history'])
        
        # Комбинируем факторы для оценки надежности наблюдения
        observation_confidence = (
            iou_consistency * 0.4 +
            linear_confidence * 0.3 +
            velocity_confidence * 0.3
        )
        
        return max(0.1, min(1.0, observation_confidence))  # Ограничиваем диапазон

    def _evaluate_motion_linearity(self, points):
        """
        Оценка линейности движения по истории точек
        Возвращает значение от 0 (нелинейное) до 1 (линейное)
        """
        if len(points) < 3:
            return 1.0
        
        # Рассчитываем векторы движения между точками
        vectors = []
        for i in range(1, len(points)):
            dx = points[i][0] - points[i-1][0]
            dy = points[i][1] - points[i-1][1]
            vectors.append((dx, dy))
        
        # Если движение практически отсутствует
        if all(abs(dx) < 2 and abs(dy) < 2 for dx, dy in vectors):
            return 1.0
        
        # Оцениваем последовательность углов между векторами
        angle_consistency = []
        for i in range(1, len(vectors)):
            v1 = vectors[i-1]
            v2 = vectors[i]
            dot_product = v1[0]*v2[0] + v1[1]*v2[1]
            norm1 = math.sqrt(v1[0]**2 + v1[1]**2)
            norm2 = math.sqrt(v2[0]**2 + v2[1]**2)
            
            if norm1 > 0.1 and norm2 > 0.1:
                cos_angle = dot_product / (norm1 * norm2)
                angle_consistency.append(abs(cos_angle))
        
        if not angle_consistency:
            return 0.8
        
        # Средняя консистентность угла
        avg_consistency = sum(angle_consistency) / len(angle_consistency)
        
        # Если движение стабильно линейное
        if avg_consistency > 0.9:
            return 1.0
        # Если движение резко меняется (нелинейное)
        elif avg_consistency < 0.5:
            return 0.3
        # Промежуточные значения
        else:
            return 0.6 + (avg_consistency - 0.5) * 0.8

    def _evaluate_velocity_stability(self, velocity_history):
        """
        Оценка стабильности скорости по истории
        Возвращает значение от 0 (нестабильная) до 1 (стабильная)
        """
        if len(velocity_history) < 3:
            return 1.0
        
        # Вычисляем изменения скорости
        accelerations = []
        for i in range(1, len(velocity_history)):
            prev_v = velocity_history[i-1]
            curr_v = velocity_history[i]
            acceleration = (
                curr_v[0] - prev_v[0],
                curr_v[1] - prev_v[1]
            )
            accelerations.append(math.sqrt(acceleration[0]**2 + acceleration[1]**2))
        
        if not accelerations:
            return 1.0
        
        # Среднее и стандартное отклонение ускорений
        mean_accel = sum(accelerations) / len(accelerations)
        if mean_accel < 1.0:  # Минимальное ускорение
            return 1.0
        
        # Коэффициент вариации (относительное стандартное отклонение)
        if len(accelerations) > 1:
            std_accel = np.std(accelerations)
            cv = std_accel / mean_accel if mean_accel > 0 else 1.0
            stability = max(0.2, 1.0 - cv)
        else:
            stability = 0.8
        
        return stability

    def observation_centric_reupdate(self, track, detection, observation_confidence):
        """
        Observation-Centric Re-Update (ORU) механизм
        Динамически корректирует состояние трека на основе надежности наблюдения
        """
        # Получаем текущее состояние
        current_bbox = track['bbox']
        detection_box = detection[0]
        
        # Вес обновления зависит от надежности наблюдения
        # При высокой надежности - больше доверяем наблюдению
        # При низкой надежности - больше доверяем предсказанию
        update_weight = min(1.0, observation_confidence * 1.5)
        
        # Плавное обновление bbox
        new_bbox = [
            current_bbox[0] * (1 - update_weight) + detection_box[0] * update_weight,
            current_bbox[1] * (1 - update_weight) + detection_box[1] * update_weight,
            current_bbox[2] * (1 - update_weight) + detection_box[2] * update_weight,
            current_bbox[3] * (1 - update_weight) + detection_box[3] * update_weight
        ]
        
        # Обновление эмбеддинга с адаптивным весом
        current_embedding = track['embedding']
        detection_embedding = detection[1]
        embedding_weight = min(1.0, observation_confidence * 0.8)  # Менее агрессивное обновление для эмбеддингов
        
        new_embedding = (
            current_embedding * (1 - embedding_weight) + 
            detection_embedding * embedding_weight
        )
        
        # Обновление скорости
        center = ((new_bbox[0] + new_bbox[2]) // 2, (new_bbox[1] + new_bbox[3]) // 2)
        prev_center = ((current_bbox[0] + current_bbox[2]) // 2, (current_bbox[1] + current_bbox[3]) // 2)
        
        if 'velocity' not in track:
            track['velocity'] = (0, 0)
            track['velocity_history'] = deque(maxlen=10)
        
        # Вычисление новой скорости
        velocity = (
            (center[0] - prev_center[0]) * self.frame_rate,
            (center[1] - prev_center[1]) * self.frame_rate
        )
        
        # Сглаживание скорости
        velocity_smoothing = 0.7 if observation_confidence > 0.7 else 0.4
        smoothed_velocity = (
            track['velocity'][0] * (1 - velocity_smoothing) + velocity[0] * velocity_smoothing,
            track['velocity'][1] * (1 - velocity_smoothing) + velocity[1] * velocity_smoothing
        )
        
        # Обновление состояния трека
        track['bbox'] = new_bbox
        track['embedding'] = new_embedding
        track['velocity'] = smoothed_velocity
        track['velocity_history'].append(velocity)
        
        # Обновление предсказанного состояния для следующего кадра
        track['predicted_bbox'] = self._predict_next_bbox(new_bbox, smoothed_velocity)
        
        return track

    def _predict_next_bbox(self, current_bbox, velocity):
        """
        Предсказание следующего положения bbox на основе скорости
        """
        center = ((current_bbox[0] + current_bbox[2]) // 2, (current_bbox[1] + current_bbox[3]) // 2)
        width = current_bbox[2] - current_bbox[0]
        height = current_bbox[3] - current_bbox[1]
        
        # Предсказание следующего положения центра
        next_center = (
            center[0] + velocity[0] / self.frame_rate,
            center[1] + velocity[1] / self.frame_rate
        )
        
        # Предсказанный bbox
        next_bbox = [
            next_center[0] - width / 2,
            next_center[1] - height / 2,
            next_center[0] + width / 2,
            next_center[1] + height / 2
        ]
        
        return next_bbox

    def update(self, detections):
        self.frame_count += 1
        
        # Если нет детекций, просто обновляем состояние треков
        if not detections:
            for track in self.tracks:
                track['time_since_update'] += 1
                track['age'] += 1
                
                # Предсказание следующего положения
                if 'velocity' in track:
                    track['bbox'] = self._predict_next_bbox(
                        track['bbox'], 
                        track['velocity']
                    )
            
            # Удаляем старые треки
            self.tracks = [t for t in self.tracks 
                          if t['time_since_update'] <= self.max_age and t['hits'] >= 1]
            return [(t['track_id'], t['bbox'], t['class_id']) for t in self.tracks 
                   if t['hits'] >= 1 and t['time_since_update'] == 0]
        
        current_boxes = [det[0] for det in detections]
        current_embeddings = [det[1] for det in detections]
        current_classes = [det[2] for det in detections]
        
        # Используем ВСЕ треки для сопоставления
        all_tracks = self.tracks.copy()
        cost_matrix = np.zeros((len(all_tracks), len(current_boxes)))
        
        # Заполнение матрицы стоимостей с учетом Observation-Centric подхода
        for i, track in enumerate(all_tracks):
            for j, (box, emb) in enumerate(zip(current_boxes, current_embeddings)):
                # Базовые метрики
                emb_sim = cosine_similarity(track['embedding'], emb)
                iou = calculate_iou(track['bbox'], box)
                
                # Дополнительные метрики для Observation-Centric подхода
                velocity_sim = 1.0
                if 'velocity' in track:
                    velocity_sim = calculate_velocity_similarity(
                        track['bbox'], box, track['velocity'], self.frame_rate
                    )
                
                # Оценка надежности наблюдения
                obs_confidence = self.estimate_observation_confidence(track, box)
                
                # Комбинированная метрика с адаптивными весами
                # При низкой надежности наблюдения - больше веса внешнему виду
                # При высокой надежности - больше веса пространственным метрикам
                if obs_confidence < 0.4:
                    combined_score = (0.8 * emb_sim + 0.2 * iou)
                else:
                    combined_score = (
                        0.5 * emb_sim + 
                        0.3 * iou + 
                        0.2 * velocity_sim
                    )
                
                # Штраф за несовпадение классов
                if track['class_id'] != current_classes[j]:
                    combined_score *= 0.5
                
                # Учет надежности наблюдения в финальном скоре
                combined_score *= obs_confidence
                
                cost_matrix[i, j] = 1.0 - combined_score if combined_score > 0.2 else 10.0
        
        # Решение задачи о назначениях
        matched_tracks = set()
        matched_detections = set()
        
        if cost_matrix.size > 0 and len(all_tracks) > 0 and len(current_boxes) > 0:
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            
            for i, j in zip(row_ind, col_ind):
                if i < len(all_tracks) and j < len(current_boxes):
                    cost = cost_matrix[i, j]
                    track = all_tracks[i]
                    detection = (current_boxes[j], current_embeddings[j], current_classes[j])
                    
                    # Порог для сопоставления зависит от надежности наблюдения
                    obs_confidence = self.estimate_observation_confidence(track, current_boxes[j])
                    dynamic_threshold = 1.0 - (self.embedding_threshold * obs_confidence * 1.2)
                    
                    if cost < dynamic_threshold:
                        # Observation-Centric Re-Update
                        track = self.observation_centric_reupdate(track, detection, obs_confidence)
                        
                        # Обновление счетчиков
                        track['hits'] += 1
                        track['age'] += 1
                        track['time_since_update'] = 0
                        track['class_id'] = current_classes[j]
                        
                        # Обновление траектории
                        center = (int((track['bbox'][0] + track['bbox'][2]) // 2),
                                 int((track['bbox'][1] + track['bbox'][3]) // 2))
                        self.trajectories[track['track_id']].append(center)
                        
                        matched_tracks.add(i)
                        matched_detections.add(j)
        
        # Обработка несопоставленных треков
        for i, track in enumerate(all_tracks):
            if i not in matched_tracks:
                track['time_since_update'] += 1
                track['age'] += 1
                
                # Предсказание следующего положения для несопоставленных треков
                if 'velocity' in track:
                    track['bbox'] = self._predict_next_bbox(
                        track['bbox'], 
                        track['velocity']
                    )
                
                # Перемещение в lost_tracks при длительном отсутствии
                if track['time_since_update'] > self.max_age // 2:
                    self.lost_tracks.append({
                        'track_id': track['track_id'],
                        'bbox': track['bbox'],
                        'embedding': track['embedding'],
                        'class_id': track['class_id'],
                        'age': track['age'],
                        'lost_frame': self.frame_count,
                        'velocity': track.get('velocity', (0, 0)),
                        'trajectory': list(self.trajectories[track['track_id']])
                    })
        
        # Создание новых треков для несопоставленных детекций
        for j, (box, emb, cls_id) in enumerate(zip(current_boxes, current_embeddings, current_classes)):
            if j not in matched_detections:
                # Предсказание начальной скорости на основе окружающих треков
                initial_velocity = self._estimate_initial_velocity(box)
                
                track_id = self.next_id
                self.next_id += 1
                
                new_track = {
                    'track_id': track_id,
                    'bbox': box,
                    'embedding': emb,
                    'class_id': cls_id,
                    'hits': 1,
                    'age': 1,
                    'time_since_update': 0,
                    'velocity': initial_velocity,
                    'velocity_history': deque(maxlen=10),
                }
                
                # Инициализация предсказанного состояния
                new_track['predicted_bbox'] = self._predict_next_bbox(box, initial_velocity)
                
                self.tracks.append(new_track)
                
                # Обновление траектории
                center = ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)
                self.trajectories[track_id].append(center)
        
        # Восстановление треков из lost_tracks
        self._recover_tracks(detections)
        
        # Удаляем старые треки
        self.tracks = [t for t in self.tracks 
                      if t['time_since_update'] <= self.max_age and t['hits'] >= 1]
        
        # Очистка старых lost_tracks
        self.lost_tracks = [t for t in self.lost_tracks 
                           if self.frame_count - t['lost_frame'] <= self.recovery_window]
        
        # Возвращаем активные треки
        return [(t['track_id'], t['bbox'], t['class_id']) for t in self.tracks 
               if t['hits'] >= self.min_hits and t['time_since_update'] == 0]

    def _estimate_initial_velocity(self, bbox):
        """
        Оценка начальной скорости для нового трека на основе окружающих треков
        """
        if not self.tracks:
            return (0, 0)
        
        # Находим ближайшие треки
        nearby_tracks = []
        center = ((bbox[0] + bbox[2]) // 2, (bbox[1] + bbox[3]) // 2)
        
        for track in self.tracks:
            if track['time_since_update'] == 0:  # Только активные треки
                track_center = ((track['bbox'][0] + track['bbox'][2]) // 2, 
                               (track['bbox'][1] + track['bbox'][3]) // 2)
                distance = math.sqrt(
                    (center[0] - track_center[0])**2 + 
                    (center[1] - track_center[1])**2
                )
                if distance < 100:  # Порог расстояния для "близких" треков
                    nearby_tracks.append(track)
        
        if not nearby_tracks:
            return (0, 0)
        
        # Средневзвешенная скорость ближайших треков
        total_weight = 0
        velocity_sum = [0, 0]
        
        for track in nearby_tracks:
            distance = math.sqrt(
                (center[0] - ((track['bbox'][0] + track['bbox'][2]) // 2))**2 + 
                (center[1] - ((track['bbox'][1] + track['bbox'][3]) // 2))**2
            )
            weight = 1.0 / max(1.0, distance)  # Ближайшие треки имеют больший вес
            
            total_weight += weight
            velocity_sum[0] += track['velocity'][0] * weight
            velocity_sum[1] += track['velocity'][1] * weight
        
        if total_weight > 0:
            avg_velocity = (
                velocity_sum[0] / total_weight,
                velocity_sum[1] / total_weight
            )
            return avg_velocity
        
        return (0, 0)

    def _recover_tracks(self, detections):
        """
        Восстановление треков из lost_tracks при обнаружении подходящих детекций
        """
        if not self.lost_tracks or not detections:
            return
        
        current_boxes = [det[0] for det in detections]
        current_embeddings = [det[1] for det in detections]
        current_classes = [det[2] for det in detections]
        
        # Матрица для сопоставления lost_tracks с детекциями
        recovery_cost = np.zeros((len(self.lost_tracks), len(current_boxes)))
        
        for i, lost_track in enumerate(self.lost_tracks):
            for j, (box, emb) in enumerate(zip(current_boxes, current_embeddings)):
                # Базовые метрики
                emb_sim = cosine_similarity(lost_track['embedding'], emb)
                iou = calculate_iou(lost_track['bbox'], box)
                
                # Дополнительные метрики
                velocity_sim = 1.0
                if 'velocity' in lost_track:
                    velocity_sim = calculate_velocity_similarity(
                        lost_track['bbox'], box, lost_track['velocity'], self.frame_rate
                    )
                
                # Комбинированная метрика для восстановления
                recovery_score = (
                    0.5 * emb_sim + 
                    0.3 * iou + 
                    0.2 * velocity_sim
                )
                
                # Дополнительный штраф за время отсутствия
                time_penalty = max(0.5, 1.0 - (self.frame_count - lost_track['lost_frame']) / self.recovery_window)
                recovery_score *= time_penalty
                
                # Штраф за несовпадение классов
                if lost_track['class_id'] != current_classes[j]:
                    recovery_score *= 0.3
                
                recovery_cost[i, j] = 1.0 - recovery_score if recovery_score > 0.4 else 10.0
        
        # Решение задачи о восстановлении
        recovered_indices = set()
        used_detections = set()
        
        if recovery_cost.size > 0:
            row_ind, col_ind = linear_sum_assignment(recovery_cost)
            
            for i, j in zip(row_ind, col_ind):
                if i < len(self.lost_tracks) and j < len(current_boxes):
                    cost = recovery_cost[i, j]
                    
                    # Порог для восстановления зависит от времени отсутствия
                    lost_track = self.lost_tracks[i]
                    time_absent = self.frame_count - lost_track['lost_frame']
                    recovery_threshold = 0.6 * (1.0 - time_absent / self.recovery_window)
                    
                    if cost < recovery_threshold and j not in used_detections:
                        # Восстанавливаем трек
                        track_id = lost_track['track_id']
                        new_track = {
                            'track_id': track_id,
                            'bbox': current_boxes[j],
                            'embedding': current_embeddings[j],
                            'class_id': current_classes[j],
                            'hits': 1,
                            'age': lost_track['age'] + 1,
                            'time_since_update': 0,
                            'velocity': lost_track.get('velocity', (0, 0)),
                            'velocity_history': deque(maxlen=10),
                        }
                        
                        # Восстанавливаем историю скоростей
                        if 'velocity' in lost_track:
                            new_track['velocity_history'].append(lost_track['velocity'])
                        
                        # Восстанавливаем траекторию
                        if track_id not in self.trajectories:
                            self.trajectories[track_id] = deque(maxlen=self.trajectory_length)
                            for point in lost_track['trajectory']:
                                self.trajectories[track_id].append(point)
                        
                        self.tracks.append(new_track)
                        
                        recovered_indices.add(i)
                        used_detections.add(j)
        
        # Удаляем восстановленные треки из lost_tracks
        self.lost_tracks = [t for i, t in enumerate(self.lost_tracks) if i not in recovered_indices]


In [ ]:
def track_objects(video_path, output_path=None):
    # Проверка существования видео
    if not os.path.exists(video_path):
        print(f"Ошибка: Файл видео не найден: {video_path}")
        return
    
    # Открытие видео
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Ошибка: Не удалось открыть видео")
        return
    

    # Инициализация трекера с Observation-Centric подходом
    tracker = EmbOCTracker(
        max_age=65,
        min_hits=2,
        iou_threshold=0.15,
        embedding_threshold=0.35,
        recovery_window=25,
        frame_rate=40,  
        velocity_weight=0.25,
        observation_confidence_alpha=0.65
    )
    
    # Параметры видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Запись результата
    output_writer = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"Начата обработка видео: {video_path}")
    print(f"Размер кадра: {width}x{height}, FPS: {fps}")
    
    frame_count = 0
    total_objects = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Детекция объектов
        results = model.predict(frame, conf=0.3, verbose=False)
        
        # Сбор детекций
        detections = []
        for result in results:
            if not hasattr(result, 'boxes') or result.boxes is None or len(result.boxes) == 0:
                continue
            
            try:
                boxes = result.boxes.xyxy.cpu().numpy()
                classes = result.boxes.cls.cpu().numpy()
                
                for i in range(len(boxes)):
                    x1, y1, x2, y2 = map(int, boxes[i])
                    cls_id = int(classes[i])
                    
                    # Проверка корректности координат
                    if x2 <= x1 or y2 <= y1:
                        continue
                    
                    # Извлекаем ROI и получаем эмбеддинг
                    roi = frame[y1:y2, x1:x2]
                    if roi.size == 0:
                        continue
                    
                    processed_roi = preprocess_roi(roi)
                    if processed_roi is None:
                        continue
                    
                    try:
                        embedding = efficientnet.predict(processed_roi[np.newaxis, ...], verbose=0).flatten()
                        detections.append(((x1, y1, x2, y2), embedding, cls_id))
                    except Exception as e:
                        print(f"Ошибка получения эмбеддинга: {e}")
                        continue
            except Exception as e:
                print(f"Ошибка обработки результата детекции: {e}")
                continue
        
        # Обновление трекера с использованием Observation-Centric подхода
        tracks = tracker.update(detections)
        
        # Обновление счетчика общего количества объектов
        if tracks:
            total_objects = max(total_objects, max([t[0] for t in tracks]))
        
        # Визуализация треков
        for track_id, bbox, class_id in tracks:
            x1, y1, x2, y2 = map(int, bbox)
            class_name = class_names.get(class_id, f'class{class_id}')
            
            # Цвет трека на основе ID
            color = (int(track_id * 30 % 255), 
                    int((track_id * 50 + 50) % 255), 
                    int((track_id * 70 + 100) % 255))
            
            # Рисуем bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Рисуем подпись: имя класса + ID трека + надежность наблюдения
            if track_id in tracker.trajectories:
                track = next((t for t in tracker.tracks if t['track_id'] == track_id), None)
                if track:
                    obs_confidence = tracker.estimate_observation_confidence(track, bbox) if 'predicted_bbox' in track else 1.0
                    label = f"{class_name} ID:{track_id} CF:{obs_confidence:.2f}"
                else:
                    label = f"{class_name} ID:{track_id}"
                
                cv2.putText(frame, label, (x1, y1 - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Рисуем траекторию
            if track_id in tracker.trajectories and len(tracker.trajectories[track_id]) > 1:
                points = list(tracker.trajectories[track_id])
                for j in range(1, len(points)):
                    thickness = max(1, 3 - j // 10)  # Затухание толщины
                    cv2.line(frame, points[j-1], points[j], color, thickness)
        
        # Отображаем визуальные индикаторы нелинейного движения
        for track in tracker.tracks:
            if 'velocity_history' in track and len(track['velocity_history']) > 5:
                # Оцениваем нелинейность движения
                linearity = tracker._evaluate_motion_linearity(list(tracker.trajectories[track['track_id']]))
                if linearity < 0.6:  
                    cx = int(round((track['bbox'][0] + track['bbox'][2]) / 2))
                    cy = int(round((track['bbox'][1] + track['bbox'][3]) / 2))
                    cv2.circle(frame, (cx, cy), 8, (0, 0, 255), 2)
        
        # Отображаем отладочную информацию
        cv2.putText(frame, f"Frame: {frame_count}/{int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Det: {len(detections)} | Trk: {len(tracks)} | Tot: {total_objects}", (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Запись и отображение
        if output_writer:
            output_writer.write(frame)
        
        cv2.imshow('Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
       
    # Очистка ресурсов
    cap.release()
    if output_writer:
        output_writer.release()
    cv2.destroyAllWindows()


Запуск трекинга

In [ ]:
video_path = "C:/ИТМО/НИР 3/data/TestVideo/UAV and hands 2.mp4"
output_path = "UAV and hands full yolo 2.mp4"
    
track_objects(video_path, output_path)

EfficientNetV2Backbone загружена
Начата обработка видео: C:/ИТМО/НИР 3/data/TestVideo/UAV and hands 2.mp4
Размер кадра: 720x1280, FPS: 30.0
